# Example of using the Borehole3D class

## Imports

In [ ]:
from striplog import Component, Decor, Legend, Lexicon, Striplog
from core.omf import Borehole3D, striplog_legend_to_omf_legend 
from utils.lexicon import Lexicon_FR_updated as lexicon_fr
from utils.io import boreholes_from_files, striplog_from_text,gdf_viewer,\
df_from_sources, striplog_from_df
from core.core import Project
from core.orm import BoreholeOrm, PositionOrm, Base 

import matplotlib.pyplot as plt
from os import remove
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR

## Reading of CSV files in a dir

In [ ]:
df = pd.read_csv('test_data_boreholes.csv', sep=',')

## Data Checking

In [ ]:
df.drop('Z', axis='columns', inplace=True)
df.rename({'new_Z':'Z'}, axis='columns', inplace=True)

In [ ]:
df.loc[df.query('Colour=="tan"').index, 'Colour'] = 'green' # use simple colour

In [ ]:
df[:10]

In [ ]:
df.query('X==152013.0 and Y==121905.0')

## Create Boreholes from dataframes or files

In [ ]:
bhs_df=[df] # list of dataframes

In [ ]:
boreholes, components, link_intv_comp = boreholes_from_files(bhs_df, verbose=True,
                                                litho_field='Lithology',
                                                color_field='Colour',
                                                lexicon='en', #Lexicon.default(),
                                            diam_field='Diameter', 
                                            thick_field='Thickness')

In [ ]:
boreholes[170].intervals[850].top.x, boreholes[170].intervals[850].top.y

In [ ]:
boreholes

In [ ]:
components

In [ ]:
len(boreholes), len(components)

## Create a project and save data in a database

In [ ]:
remove('tmp_files/tfe_orm_db.db')

In [ ]:
engine = create_engine('sqlite:///tmp_files/tfe_orm_db.db', echo=True)

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
p = Project(session)
p.add_components(components)

Add boreholes into the project

In [ ]:
for bh in boreholes:
    p.add_borehole(bh)

In [ ]:
p.add_link_between_components_and_intervals(link_intv_comp)

In [ ]:
p.commit()
print('Boreholes in the project : ', len(p.boreholes))

In [ ]:
p.boreholes

In [ ]:
p.refresh(verbose=True)

In [ ]:
session.close()

# 3D Display

## Display one object

In [ ]:
bh = p.boreholes_3d[0]
print(bh.name, '\n', bh.intervals) # problem with top and base position --> kind of overlaying

In [ ]:
p.boreholes_3d[0].plot2d()

In [ ]:
p.boreholes_3d[0].plot3d()

In [ ]:
p.boreholes_3d[0].plot3d(x3d=True)

### Display project boreholes

In [ ]:
p.plot3d(labels_size=15, bg_color=["royalblue", "aliceblue"], window_size=(1300, 1000))

### Boreholes in 3D using X3D for interaction within the notebook